In [1]:
from typing import Tuple
import pandas

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-hackathon23_somnolence/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
df = pandas.read_excel(io = '../../data/processed/france.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    df_covid_holidays = pandas.DataFrame()
    return df_mrd, df_covid_holidays

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
model = Prophet().fit(df_train)

00:01:08 - cmdstanpy - INFO - Chain [1] start processing
00:01:09 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
def test_model(df_test: pandas.DataFrame, model):
    "return predicted values and rmse"
    df_predicted = model.predict(df_test)
    rmse = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = True)
    return df_predicted, rmse

In [9]:
df_predicted, rmse = test_model(df_test, model)

In [10]:
rmse

0.06700005152690783

In [11]:
with open('france_prophet_benchmark_model.json', 'w') as f:
    f.write(model_to_json(model))